<a href="https://colab.research.google.com/github/annab0503/DS4002/blob/main/Project%201/Analysis%20Data/Visa_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import dependencies
import pandas as pd
import os
import requests

### Step 1: Load the Excel file from the URL

In [2]:
url = 'https://github.com/annab0503/DS4002/blob/main/Project%201/Original%20Data/FYs97-14_NIVDetailTable.xls?raw=true'
response = requests.get(url)
excel_file_path = '/content/FYs97-14_NIVDetailTable.xls'

# Save the file locally
with open(excel_file_path, 'wb') as f:
    f.write(response.content)

# Load the Excel file
excel_data = pd.ExcelFile(excel_file_path)

# Directory to save the CSV files
csv_directory = '/content/csv_files'
os.makedirs(csv_directory, exist_ok=True)  # Ensure directory exists

### Step 2: Iterate through each sheet in the Excel file and save as CSV

In [3]:
for sheet_name in excel_data.sheet_names:
    # Load the sheet into a DataFrame
    df = excel_data.parse(sheet_name)

    # Define the output CSV file name
    csv_file = os.path.join(csv_directory, f'{sheet_name}.csv')

    # Save the DataFrame as a CSV file
    df.to_csv(csv_file, index=False)
    print(f"Saved sheet '{sheet_name}' to {csv_file}")

Saved sheet 'FY97' to /content/csv_files/FY97.csv
Saved sheet 'FY98' to /content/csv_files/FY98.csv
Saved sheet 'FY99' to /content/csv_files/FY99.csv
Saved sheet 'FY00' to /content/csv_files/FY00.csv
Saved sheet 'FY01' to /content/csv_files/FY01.csv
Saved sheet 'FY02' to /content/csv_files/FY02.csv
Saved sheet 'FY03' to /content/csv_files/FY03.csv
Saved sheet 'FY04' to /content/csv_files/FY04.csv
Saved sheet 'FY05' to /content/csv_files/FY05.csv
Saved sheet 'FY06' to /content/csv_files/FY06.csv
Saved sheet 'FY07' to /content/csv_files/FY07.csv
Saved sheet 'FY08' to /content/csv_files/FY08.csv
Saved sheet 'FY09' to /content/csv_files/FY09.csv
Saved sheet 'FY10' to /content/csv_files/FY10.csv
Saved sheet 'FY11' to /content/csv_files/FY11.csv
Saved sheet 'FY12' to /content/csv_files/FY12.csv
Saved sheet 'FY13' to /content/csv_files/FY13.csv
Saved sheet 'FY14' to /content/csv_files/FY14.csv


### Step 3: Process each CSV file

In [4]:

csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

# Loop over each CSV file for processing
for csv_file in csv_files:
    # Load the CSV file into a DataFrame
    year_data = pd.read_csv(os.path.join(csv_directory, csv_file))

    # Drop empty rows
    year_data = year_data.dropna(how='all')

    # Rename the 'Fiscal Year' column to 'Country'
    fiscal_year_column = next((col for col in year_data.columns if 'Fiscal Year' in col), None)
    if fiscal_year_column:
        year_data.rename(columns={fiscal_year_column: 'Country'}, inplace=True)

    # Remove rows for continents and totals
    countries_to_drop = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'Unknown']
    year_data = year_data[~year_data['Country'].isin(countries_to_drop)]
    year_data = year_data[~year_data['Country'].str.contains('Totals', na=False)]

    # Reshape the DataFrame with 'melt'
    year_data = year_data.melt(id_vars=['Country'],
                                var_name='Type of U.S. Visa',
                                value_name='Quantity of U.S Visas Granted')

    # Extract year from the filename (e.g., 'FY1997.csv')
    year = int(csv_file.split('FY')[1].split('.csv')[0])
    year_data['Fiscal Year'] = year

    # Save the reshaped DataFrame as a new CSV file
    output_file = os.path.join(csv_directory, f"processed_{csv_file}")
    year_data.to_csv(output_file, index=False)

    print(f"Processed {csv_file} and saved to {output_file}")

Processed FY11.csv and saved to /content/csv_files/processed_FY11.csv
Processed FY09.csv and saved to /content/csv_files/processed_FY09.csv
Processed FY08.csv and saved to /content/csv_files/processed_FY08.csv


<ipython-input-4-613bd8103da5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_data.rename(columns={fiscal_year_column: 'Country'}, inplace=True)


Processed FY07.csv and saved to /content/csv_files/processed_FY07.csv
Processed FY04.csv and saved to /content/csv_files/processed_FY04.csv
Processed FY13.csv and saved to /content/csv_files/processed_FY13.csv
Processed FY03.csv and saved to /content/csv_files/processed_FY03.csv
Processed FY99.csv and saved to /content/csv_files/processed_FY99.csv
Processed FY01.csv and saved to /content/csv_files/processed_FY01.csv
Processed FY02.csv and saved to /content/csv_files/processed_FY02.csv
Processed FY10.csv and saved to /content/csv_files/processed_FY10.csv
Processed FY00.csv and saved to /content/csv_files/processed_FY00.csv
Processed FY98.csv and saved to /content/csv_files/processed_FY98.csv
Processed FY06.csv and saved to /content/csv_files/processed_FY06.csv
Processed FY14.csv and saved to /content/csv_files/processed_FY14.csv
Processed FY05.csv and saved to /content/csv_files/processed_FY05.csv
Processed FY12.csv and saved to /content/csv_files/processed_FY12.csv
Processed FY97.csv a

### Step 4: Merge all processed CSV files into one DataFrame

In [5]:
processed_csv_files = [f for f in os.listdir(csv_directory) if f.startswith('processed_') and f.endswith('.csv')]

# List to hold DataFrames for merging
dfs = []

# Load each processed CSV file into a DataFrame and append to the list
for csv_file in processed_csv_files:
    df = pd.read_csv(os.path.join(csv_directory, csv_file))
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_csv_file = os.path.join(csv_directory, 'merged_data.csv')
merged_df.to_csv(merged_csv_file, index=False)

print(f"All processed CSV files merged and saved to {merged_csv_file}")

All processed CSV files merged and saved to /content/csv_files/merged_data.csv


 ### Step 5: Load the merged data

In [7]:
visa_data = pd.read_csv('/content/csv_files/merged_data.csv')

# Convert 'Year' column to numeric, coercing errors to NaN
visa_data['Fiscal Year'] = pd.to_numeric(visa_data['Fiscal Year'], errors='coerce')

# Check and display the number of non-numeric values in 'Year'
print(visa_data['Fiscal Year'].isna().sum(), "non-numeric values converted to NaN")

0 non-numeric values converted to NaN


### Step 6: Map fiscal year codes to actual years

In [8]:
code_to_year_map = {
    97: 1997, 98: 1998, 99: 1999, 0: 2000, 1: 2001, 2: 2002, 3: 2003, 4: 2004, 5: 2005,
    6: 2006, 7: 2007, 8: 2008, 9: 2009, 10: 2010, 11: 2011, 12: 2012, 13: 2013, 14: 2014
}

# Apply the mapping to the 'Year' column to convert codes to actual years
visa_data['Fiscal Year'] = visa_data['Fiscal Year'].map(code_to_year_map)

# Save the cleaned and merged data to a new CSV file
visa_data.to_csv('visa_data.csv', index=False)